In [4]:
import torch
import json
import re
torch.__version__

'1.8.0'

In [12]:
def reader(file):
    data = []
    with open(f'../data/{file}', 'r') as file:
        for line in file.readlines():
            data.append(json.loads(line))
    return data

In [14]:
def token(line):
    

[{'verified': True,
  'reviewTime': '09 11, 2013',
  'reviewerID': 'AC2PL52NKPL29',
  'asin': '0001388703',
  'style': {'Format:': ' Audio CD'},
  'reviewText': 'So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!',
  'summary': 'Gotta listen to this!',
  'unixReviewTime': 1378857600,
  'sentiment': 'positive',
  'id': 0},
 {'verified': False,
  'reviewTime': '05 14, 2009',
  'reviewerID': 'A3NVGWKHLULDHR',
  'asin': '0001388703',
  'style': {'Format:': ' Audio Cassette'},
  'reviewText': 'This tape can hardly be understood and it was listed for sale as "very good".  It\'s VERY BAD.',
  'summary': 'Shame Shame',
  'unixReviewTime': 1242259200,
  'sentiment': 'negative',
  'id': 1},
 {'verified': True,
  'reviewTime': '05 15, 2015',
  'reviewerID': 'AZ3T21W6CW0MW',
  'asin': '0001388703',
  'style': {'Format:': ' MP3 Music'},
  'reviewText': "Buy the CD.  Do not buy the MP3 album.  Download is no longer availabl